In [1]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\provo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\provo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\provo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
train_data = np.load('data_train.pkl', allow_pickle=True)
test_data = np.load('data_test.pkl', allow_pickle=True)

In [23]:
class NaiveBayes:
    def __init__(self):
        self.tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')    # https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer
        self.wnl = nltk.stem.WordNetLemmatizer()
    
    def train(self, data):
        # Distinguish comments and labels
        comments = data[0]
        labels = np.array(data[1])
        unique_labels = np.unique(labels)
        
        # Initialize list of bags
        self.bags = []

        # Iterate over all classes
        j = 0
        for label in unique_labels:
            s = ''
            # Find all entries for that class
            indices = np.where(labels == label)[0]
            # Iterate over those entries
            for ind in indices:
                # Add comment to string
                s += str(comments[ind])
            
            # Split string into words
            s = self.tokenizer.tokenize(s)
            
            # Remove stop words
            s = [w for w in s if w not in stopwords.words('english')]
            
            # Convert words to their base form
            s = [self.wnl.lemmatize(w) for w in s]
            
            # Count word frequency and calculate log probabilities
            words, counts = np.unique(s, return_counts=True)
            probs = np.log(counts+1 / np.sum(counts+1))
            
            # Create bag of words
            self.bags.append((words,counts,probs))
            
            # Print progress
            j+=1
            print(str(j) + '/20', end='\r')
    
    def predict(self, data):
        # Initialize list of predictions
        predictions = []
        j = 1
        
        # Iterate over comments
        for comment in data:
            # Split comment using tokenizer
            comment = self.tokenizer.tokenize(comment)

            # Initialize list of class probabilities
            self.probabilities = [0 for i in range(20)]
            
            # Iterate over bags of words
            for prob_idx, bag in enumerate(self.bags):
                # Iterate over words in 
                for word in comment:
                    # Find index of word in bag (if possible)
                    bag_idx = np.where(bag[0] == word)[0]
                    
                    # Add word probability to class probability
                    if bag_idx.size > 0:
                        self.probabilities[prob_idx] += bag[2][bag_idx][0]
                    else:
                        self.probabilities[prob_idx] += np.log(1/(np.sum(bag[1]+1)))
                        
            # Predict class with highest probability
            predictions.append(np.argmax(self.probabilities))
            
            # Print progress
            print(str(j) + '/' + str(len(data)), end='\r')
            j += 1
            
        return predictions

In [13]:
def make_csv(predictions):
    f = open('submission.csv', 'w')
    f.close()
    f = open('submission.csv', 'a')
    f.write('Id,Category\n')
    for idx, pred in enumerate(predictions):
        f.write(str(idx) + ',' + classes[pred] + '\n')
    pass

In [25]:
clf = NaiveBayes()

In [26]:
clf.train((train_data[0][:5000], train_data[1][:5000]))

In [29]:
predictions = clf.predict(train_data[0][-1000:])

In [31]:
classes = np.unique(train_data[1])
correct = 0
for idx, pred in enumerate(predictions):
    if classes[pred] == train_data[1][69000+idx]:
        correct += 1
print(correct/len(predictions))

0.316


In [ ]:
make_csv(predictions)

In [36]:
#0.357
np.sum([1,2,3,4])
test = np.array([1,2,3,4])
np.sum(test + 1)

14